# Creating dataset - Web Scraping

Extraer la BBDD con datos de las empresas de **`invertirenbolsa.info`**

La url general donde se encuentran los enlaces a cada empresa:

In [1]:
la_url = 'https://www.invertirenbolsa.info/historico_dividendos.htm'

Importar librerías

In [2]:
import pandas as pd
import numpy as np

from IPython.display import IFrame

Para mostrar una vista previa de la página:

In [ ]:
## IFrame(la_url, 800, 600)

Extraer la tabla con los nombres de la empresa para posteriormente utilizarlo como parte de cada url:

In [3]:
master_table = pd.read_html(la_url, header=0)

In [4]:
master_table[0].head()

,Empresa,Cotización,Dividendo,RD%,BPA ord.,PER,VCA,P / VC,Prev
0,3M,18032.0,58800,"3,26%","9,45(e)","19,08(e)",2221,812,+ x Porcentaje recomendado a largo plazo en un...
1,AB Foods,2273.0,"0,4635(e)","2,04%(e)",057,3988,1182,192,+ x Suspende el dividendo de 2020 porque en el...
2,Acciona,13000.0,19250,"1,48%",641,2028,6237,208,+ x Porcentaje recomendado a largo plazo en un...
3,Acerinox,973.0,05000,"5,14%",-022,-4423,692,141,+ x Creo que Acerinox puede ser válidad para e...
4,ACS,2646.0,19900,"7,52%",306,865,1403,189,+ x Porcentaje recomendado a largo plazo en un...


In [5]:
master_table[0].shape

(109, 9)

Extraer cada valor de la columna "Empresa" para iterar y utilizarlo en cada url:

In [6]:
# Número de empresas distintas

len(master_table[0]['Empresa'].unique())

109

In [7]:
# Valores de esas empresas

empresas = master_table[0]['Empresa'].unique()
empresas

array(['3M', 'AB Foods', 'Acciona', 'Acerinox', 'ACS', 'Adidas', 'Aena',
       'Air Liquide', 'Airbus', 'Allianz', 'Amazon', 'Archer Daniels',
       'ATT', 'Aviva', 'Axa', 'Banco Sabadell', 'Banco Santander',
       'Bankia', 'Bankinter', 'Basf', 'BBVA', 'BMW', 'Boeing', 'BP', 'BT',
       'Burberry', 'CAF', 'Caixabank', 'Catalana Occidente',
       'Caterpillar', 'Cellnex', 'Chevron', 'CIE Automotive', 'Coca Cola',
       'Colgate', 'Colonial', 'Daimler', 'Danone', 'Deutsche Telekom',
       'E.on', 'Ebro Foods', 'Elecnor', 'Enagás', 'Ence', 'Endesa',
       'Engie', 'Euronext', 'Exxon', 'Facebook', 'FCC', 'Ferrari',
       'Ferrovial', 'General Electric', 'General Mills', 'Gestamp',
       'Google (Alphabet)', 'Grifols', 'Henkel', 'Iberdrola', 'Iberpapel',
       'IBM', 'Illinois Tool Works', 'Inditex', 'Indra', 'Intel',
       'Johnson & Johnson', 'Kimberly-Clark', 'Klepierre', 'Kraft Heinz',
       'Linde', 'Lingotes Especiales', 'Logista', 'LSE', 'LVMH', 'Mapfre',
       'McDona

**Transformar nombres de empresa:**
- Agregar el nombre de la empresa al final para completar cada url:  
    - https://www.invertirenbolsa.info/historicodividendos/empresa/ ...  
- Cambiar los espacios por guión medio:  
    - *Banco Santander >> banco-santander*
- Eliminar los paréntesis:  
    - *Google (Alphabet) >> google-alphabet*
- Cambiar los puntos por guión medio:     
    - *E.on >> e-on*
- Eliminar el carácter '&':
    - *Johnson & Johnson >> johnson-johnson*
- Eliminar tildes.
- Transformar todas las letras en minúsculas.

In [8]:
# Bucle para transformar cada nombre de empresa

empresas_clean =  []

for empresa in empresas:
    empresa = empresa.replace("&", "")
    empresa = empresa.replace("(", "")
    empresa = empresa.replace(")", "")
    empresa = empresa.replace(" ", "-")
    empresa = empresa.replace(".", "-")
    empresa = empresa.replace("--", "-") # Para evitar error en Johnson & Johnson
    empresa = empresa.lower()
    empresa = empresa.replace("á", "a")
    empresa = empresa.replace("é", "e")
    empresa = empresa.replace("í", "i")
    empresa = empresa.replace("ó", "o")
    empresas_clean.append(empresa)
empresas_clean

['3m',
 'ab-foods',
 'acciona',
 'acerinox',
 'acs',
 'adidas',
 'aena',
 'air-liquide',
 'airbus',
 'allianz',
 'amazon',
 'archer-daniels',
 'att',
 'aviva',
 'axa',
 'banco-sabadell',
 'banco-santander',
 'bankia',
 'bankinter',
 'basf',
 'bbva',
 'bmw',
 'boeing',
 'bp',
 'bt',
 'burberry',
 'caf',
 'caixabank',
 'catalana-occidente',
 'caterpillar',
 'cellnex',
 'chevron',
 'cie-automotive',
 'coca-cola',
 'colgate',
 'colonial',
 'daimler',
 'danone',
 'deutsche-telekom',
 'e-on',
 'ebro-foods',
 'elecnor',
 'enagas',
 'ence',
 'endesa',
 'engie',
 'euronext',
 'exxon',
 'facebook',
 'fcc',
 'ferrari',
 'ferrovial',
 'general-electric',
 'general-mills',
 'gestamp',
 'google-alphabet',
 'grifols',
 'henkel',
 'iberdrola',
 'iberpapel',
 'ibm',
 'illinois-tool-works',
 'inditex',
 'indra',
 'intel',
 'johnson-johnson',
 'kimberly-clark',
 'klepierre',
 'kraft-heinz',
 'linde',
 'lingotes-especiales',
 'logista',
 'lse',
 'lvmh',
 'mapfre',
 'mcdonalds',
 'merlin',
 'microsoft',
 '

**Crear el dataset:**
- Agregar el nombre de la empresa al final para completar cada url y acceder a cada Tabla número 4 de la misma.
- Eliminar las filas `%`, `Observaciones` y vacías.
- Tomar las columnas hasta encontrar el primer año con `NaN`
- Intentar que no se pierdan las "comas" (como separador de decimales). **`OK!`**
- Añadir el nombre de la empresa y el año en 2 columnas, transponiendo los datos.
- Añadir también otra columna con el sector de la empresa.


In [9]:
url_inicio = 'https://www.invertirenbolsa.info/historicodividendos/empresa/'

dataset = pd.DataFrame()

for empresa in empresas_clean:
    url_complete = url_inicio + empresa
    tables = pd.read_html(url_complete, decimal = ',' , thousands='.')
    df_empresa = tables[4]
    df_empresa['Company'] = empresa
    df_empresa = df_empresa.rename(columns ={'Unnamed: 0': 'KPI'}) # Renombrar columna
 
    # Reordenar las columnas insertando 'Empresa' al principio del df
    cols = list(df_empresa)
    cols = [cols[-1]] + cols[:-1]
    df_empresa = df_empresa[cols]
    
    # Reshaping by melt
    df_empresa = df_empresa.melt(id_vars=["Company", "KPI"], var_name="Year")
    
    # Agregar al df general
    dataset = pd.concat([dataset, df_empresa])

dataset.head()

,Company,KPI,Year,value
0,3m,Ingresos,2020,32184.00
1,3m,%,2020,0.15%
2,3m,Coste de ventas,2020,-16605.00
3,3m,%,2020,3.1%
4,3m,Margen bruto,2020,15579.00


Obtener el país y el sector de cada empresa y agregarlo al dataset:

In [10]:
import requests
import re
from bs4 import BeautifulSoup

url_inicio = 'https://www.invertirenbolsa.info/historicodividendos/empresa/'

df_pais_sector = pd.DataFrame()

for empresa in empresas_clean:
    url_complete = url_inicio + empresa
    reqs = requests.get(url_complete)
    soup = BeautifulSoup(reqs.text, 'lxml')
    for heading in soup.find_all(["h3"]):
        pais_sector = (heading.text.strip())
        break
    
    # Extraer el país:
    pais_fin = pais_sector.index('.')
    pais_empresa = pais_sector[6:pais_fin]
    
    # Extraer el sector:
    sector = re.search(r'.*?Sector: (\w+).*', pais_sector)
    sector_empresa = (sector.group(1))
    
    # Construir el df:
    df_pais_sector.loc[empresa,'Company'] = empresa
    df_pais_sector.loc[empresa, 'Country'] = pais_empresa
    df_pais_sector.loc[empresa, 'Sector'] = sector_empresa

df_pais_sector.head()

,Company,Country,Sector
3m,3m,USA,Consumo
ab-foods,ab-foods,Inglaterra,Alimentación
acciona,acciona,España,Construcción
acerinox,acerinox,España,Industriales
acs,acs,España,Construcción


In [11]:
df_pais_sector.groupby(['Country'])[['Country']].count()

,Country
Country,
España,42
Inglaterra,11
USA,31
Zona Euro,25


In [12]:
df_pais_sector.groupby(['Sector'])[['Sector']].count()

,Sector
Sector,
Agua,2
Alimentación,8
Aseguradoras,6
Automóviles,5
Autopistas,2
Bancos,6
Construcción,5
Consumo,12
Distribución,2


In [13]:
# Merge:

dataset = dataset.merge(df_pais_sector, left_on = 'Company', right_on ='Company')
dataset.head()

,Company,KPI,Year,value,Country,Sector
0,3m,Ingresos,2020,32184.00,USA,Consumo
1,3m,%,2020,0.15%,USA,Consumo
2,3m,Coste de ventas,2020,-16605.00,USA,Consumo
3,3m,%,2020,3.1%,USA,Consumo
4,3m,Margen bruto,2020,15579.00,USA,Consumo


In [14]:
# Cambio de orden de las columnas:

dataset = dataset[['Company', 'Country', 'Sector', 'KPI', 'Year', 'value']]
dataset.head()

,Company,Country,Sector,KPI,Year,value
0,3m,USA,Consumo,Ingresos,2020,32184.00
1,3m,USA,Consumo,%,2020,0.15%
2,3m,USA,Consumo,Coste de ventas,2020,-16605.00
3,3m,USA,Consumo,%,2020,3.1%
4,3m,USA,Consumo,Margen bruto,2020,15579.00


Eliminar filas: `%`, `Observaciones` y vacías:

In [15]:
dataset.shape

(161373, 6)

In [16]:
dataset.isnull().sum()

Company        0
Country        0
Sector         0
KPI        12925
Year           0
value      43086
dtype: int64

In [17]:
dataset = dataset.dropna()
dataset.shape

(118287, 6)

In [18]:
dataset.isnull().sum()

Company    0
Country    0
Sector     0
KPI        0
Year       0
value      0
dtype: int64

In [19]:
dataset = dataset.drop(dataset[dataset['KPI']=='%'].index)
dataset = dataset.drop(dataset[dataset['KPI']=='Observaciones'].index)

In [20]:
dataset.shape

(90050, 6)

In [21]:
dataset.dtypes

Company    object
Country    object
Sector     object
KPI        object
Year       object
value      object
dtype: object

In [22]:
dataset['value'] = dataset['value'].replace({'%':''}, regex=True)
dataset['value'] = dataset['value'].replace({',':'.'}, regex=True)
dataset['value'] = dataset['value'].astype(float)

In [23]:
# Crear Wide Dataset

wide_dataset = dataset.pivot_table(index=["Company", 'Country', 'Sector', "Year"], 
                    columns='KPI', 
                    values='value').reset_index()
wide_dataset.head()

KPI,Company,Country,Sector,Year,Activos totales,Activos totales / fondos propios,BPA %,BPA extraordinario,BPA ordinario,BPA total,...,ROCE,ROE,Ratio combinado,Ratio de eficiencia,Resultado neto ordinario,Resultado neto total,Tasa de morosidad,Tier 1,Valor contable por acción,Variación tipos cambio
0,3m,USA,Consumo,1987,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3m,USA,Consumo,1988,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3m,USA,Consumo,1989,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3m,USA,Consumo,1990,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3m,USA,Consumo,1991,NaN,NaN,NaN,0.0,1.31,1.31,...,NaN,NaN,NaN,NaN,1154.0,NaN,NaN,NaN,NaN,-62.0


Exportar a Excel:

In [24]:
wide_dataset.to_excel(r'../data/Empresas_wide.xlsx', index=False)

In [27]:
dataset.to_excel(r'../data/Empresas.xlsx', index=False)